In [1]:
pip install pyworld

     |████████████████████████████████| 212 kB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609470 sha256=98f5f0e7009fc4b2761dcd81a61608a48fb8fed9fe5119965a5b74cae13df52b
  Stored in directory: /root/.cache/pip/wheels/e7/7c/11/c775fffa0e1e7b05a6604b4323408a77f80fb4ab304d96b5c6
Successfully built pyworld


In [2]:
pip install pysptk

     |████████████████████████████████| 410 kB 4.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pysptk: filename=pysptk-0.1.20-cp37-cp37m-linux_x86_64.whl size=929907 sha256=a7df2a622fee366633707fa9d57fd99c00309f62c38196ac379bbb6fcfae78b3
  Stored in directory: /root/.cache/pip/wheels/dd/37/3f/7583b00bf73e73650aad5d268ee46b95dbd6dffbd237e1bf58
Successfully built pysptk


**Dataset Downloaded from drive**


In [3]:
from google_drive_downloader.google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1T3R7FDi-2fgJCuxooteRt9K_W_p7CRoo',
                                    dest_path='/content/Dataset',
                                    unzip=True)

Unzipping...Done.


**Import important libraries**

In [4]:
from scipy.io import wavfile
import pysptk
from pysptk.synthesis import Synthesizer, MLSADF
import pyworld
from os.path import join, basename

#from nnmnkwii import preprocessing as P
#from nnmnkwii.paramgen import unit_variance_mlpg_matrix

#import gantts
#from hparams import vc as hp

import librosa
import librosa.display
import IPython
from IPython.display import Audio
import os
import numpy as np
import glob


**Only path to folder is needed**

In [5]:
orig_path ='/content/Male-to-Male-Voice/SM1'
synth_path = '/content/Male-to-Male-Voice/converted_SM2_Speech_in_SM1_Voice'

**All the paths are extracted into a list**

In [6]:
orig_wav_files=glob.glob(orig_path+'/*')
synth_wav_files=glob.glob(synth_path+'/*')
orig_wav_files[:]

['/content/Male-to-Male-Voice/SM1/100061.wav',
 '/content/Male-to-Male-Voice/SM1/100080.wav',
 '/content/Male-to-Male-Voice/SM1/100041.wav',
 '/content/Male-to-Male-Voice/SM1/100087.wav',
 '/content/Male-to-Male-Voice/SM1/100072.wav',
 '/content/Male-to-Male-Voice/SM1/100099.wav',
 '/content/Male-to-Male-Voice/SM1/100067.wav',
 '/content/Male-to-Male-Voice/SM1/100002.wav',
 '/content/Male-to-Male-Voice/SM1/100054.wav',
 '/content/Male-to-Male-Voice/SM1/100124.wav',
 '/content/Male-to-Male-Voice/SM1/100018.wav',
 '/content/Male-to-Male-Voice/SM1/100091.wav',
 '/content/Male-to-Male-Voice/SM1/100016.wav',
 '/content/Male-to-Male-Voice/SM1/100160.wav',
 '/content/Male-to-Male-Voice/SM1/100077.wav',
 '/content/Male-to-Male-Voice/SM1/100034.wav',
 '/content/Male-to-Male-Voice/SM1/100103.wav',
 '/content/Male-to-Male-Voice/SM1/100135.wav',
 '/content/Male-to-Male-Voice/SM1/100079.wav',
 '/content/Male-to-Male-Voice/SM1/100053.wav',
 '/content/Male-to-Male-Voice/SM1/100092.wav',
 '/content/Ma

In [7]:
from tqdm import tnrange
fs = 16000
frame_period=5.0
hop_length = int(fs * (frame_period * 0.001))
ms_fftlen = 4096
modfs = fs / hop_length
ms_freq = np.arange(ms_fftlen//2 + 1)/ms_fftlen * modfs

** Function for computing Static Features using pyworld(WORLD VOCODER)**


In [8]:
def compute_static_features(path):
    fs, x = wavfile.read(path)
    x = x.astype(np.float64)
    f0, timeaxis = pyworld.dio(x, fs, frame_period=5.0)
    f0 = pyworld.stonemask(x, f0, timeaxis, fs)
    spectrogram = pyworld.cheaptrick(x, f0, timeaxis, fs)
    aperiodicity = pyworld.d4c(x, f0, timeaxis, fs)
    alpha = pysptk.util.mcepalpha(fs)
    mc = pysptk.sp2mc(spectrogram, order=24, alpha=alpha)
    c0, mc = mc[:, 0], mc[:, 1:]
    return mc

**Finding Modulation Spectra**

In [9]:
def modspec(x, n=4096, norm=None, return_phase=False):
    
    # DFT against time axis
    s_complex = np.fft.rfft(x, n=n, axis=0, norm=norm)
    assert s_complex.shape[0] == n // 2 + 1
    R, im = s_complex.real, s_complex.imag
    ms = R * R + im * im

    # TODO: this is ugly...
    if return_phase:
        return ms, np.exp(1.0j * np.angle(s_complex))
    else:
        return ms

**Function for finding the Mean over all files**

In [10]:
def mean_modspec(path):
    mss = []
    for wav in path: 
        mgc = compute_static_features(wav)
        #print(mgc)
        #b=log(modspec(mgc, n=ms_fftlen))
        #print(b)
        #print(np.any(b<=0))

    
        #print("wav"+)
        ms = np.log(modspec(mgc, n=ms_fftlen))
        mss.append(ms)
        #return mss
        return np.mean(np.array(mss), axis=(0,))

In [11]:
ms_into2out_orig=mean_modspec(orig_wav_files)

In [12]:
ms_into2out_synth=mean_modspec(synth_wav_files)

**Calculating the Modulation Spectra Distance**

In [20]:
import math
new=0
for i in range(24):
    a=ms_into2out_orig[i, :].T
    b=ms_into2out_synth[i,:].T
    diff=np.mean(np.absolute(a-b))
    diff=(np.inner(diff, diff))
    new=new+diff


MSD=math.sqrt(1/len(mean_modspec(orig_wav_files).T))*math.sqrt(new)
print(MSD)

1.3962559058787383
